In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv


In [2]:
df = pd.read_csv("/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv")
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [3]:
df.isnull().sum()

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

In [4]:
col_names = df.columns
col_names

Index(['CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency',
       'Support Calls', 'Payment Delay', 'Subscription Type',
       'Contract Length', 'Total Spend', 'Last Interaction', 'Churn'],
      dtype='object')

In [5]:
df.dtypes

CustomerID           float64
Age                  float64
Gender                object
Tenure               float64
Usage Frequency      float64
Support Calls        float64
Payment Delay        float64
Subscription Type     object
Contract Length       object
Total Spend          float64
Last Interaction     float64
Churn                float64
dtype: object

In [6]:
df.shape

(440833, 12)

In [7]:
df = df.dropna(axis=0, how='all')

In [8]:
df.shape

(440832, 12)

In [9]:
df['Contract Length'].nunique()

3

In [10]:
df['Churn'].nunique()

2

In [11]:
df['Subscription Type'].nunique()

3

In [12]:
df.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [13]:
df['Gender'].value_counts()

Gender
Male      250252
Female    190580
Name: count, dtype: int64

In [14]:
# for col in col_names:
#     print(df[col].value_counts())

In [15]:
#detect outlier

def detect_outlier(df, column):
    q3 = df[column].quantile(0.75)
    q1 = df[column].quantile(0.25)
    iqr = q3-q1
    lower = iqr - 1.5*q1
    upper = iqr -1.5*q3
    outlier = df[(df[column] < lower) | (df[column] > upper)]
    return outlier.shape

In [16]:
detect_outlier(df, 'Tenure')

(440832, 12)

In [19]:
df['Tenure'].mean()
low-up

-30.0

In [18]:
mm = df['Tenure'].mean()
low = df['Tenure'].quantile(0.25)
up = df['Tenure'].quantile(0.75)
(up-low) - 1.5*low

6.0

In [23]:
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,0.255319,0,39.0,0.448276,0.5,0.600000,2,0,0.924444,0.551724,1.0
1,3.0,1.000000,0,49.0,0.000000,1.0,0.266667,0,1,0.507778,0.172414,1.0
2,4.0,0.787234,0,14.0,0.103448,0.6,0.600000,0,2,0.094444,0.068966,1.0
3,5.0,0.851064,1,38.0,0.689655,0.7,0.233333,2,1,0.328889,0.965517,1.0
4,6.0,0.106383,1,32.0,0.655172,0.5,0.266667,0,1,0.574444,0.655172,1.0


In [21]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Subscription Type'] = label_encoder.fit_transform(df['Subscription Type'])
df['Contract Length'] = label_encoder.fit_transform(df['Contract Length'])
# df['Churn'] = label_encoder.fit_transform(df['Churn'])

In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

columns_to_scale = ['Age', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Total Spend', 'Last Interaction']
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440832 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  int64  
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  int64  
 8   Contract Length    440832 non-null  int64  
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), int64(3)
memory usage: 43.7 MB


In [31]:
from sklearn.model_selection import train_test_split
Y = df['Churn']
X = df.drop(columns=['Churn'])

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.8, shuffle=True)

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(xtrain, ytrain)

RandomForestClassifier()

In [36]:
ypred = rf.predict(xtest)

In [40]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(ytest, ypred)
print(accuracy)

0.9993988680572096


In [38]:
ytest

200931    1.0
69012     1.0
367064    0.0
391377    0.0
9524      1.0
         ... 
29795     1.0
107885    1.0
339751    0.0
198097    1.0
391882    0.0
Name: Churn, Length: 88167, dtype: float64

In [39]:
ypred

array([1., 1., 0., ..., 0., 1., 0.])